In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from collections import namedtuple
import json
from argparse import Namespace
from neural_srl.shared.measurements import Timer
from neural_srl.shared.tagger_data import TaggerData
from neural_srl.shared import reader
import os
import shutil
import torch

/home/longvb/anaconda3/envs/deepsrl-py3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [6]:
Argv = namedtuple("Argv", "config dev gold labels model task train vocab")

In [7]:
args = Argv(config='../config/srl_small_config.json', dev='../data/srl/conll2012.devel.txt', gold='../data/srl/conll2012.devel.props.gold.txt', labels='', model='conll2012_small_model', task='srl', train='../data/srl/conll2012.train.txt', vocab='')

In [8]:
args.config

'../config/srl_small_config.json'

In [9]:
def get_config(config_filepath):
  with open(config_filepath, 'r') as config_file: 
    conf = json.load(config_file, object_hook=lambda d: Namespace(**d))
  return conf

In [10]:
config = get_config(args.config)

In [11]:
i = 0
global_step = 0
epoch = 0
train_loss = 0.0

In [12]:
with Timer('Data loading'):
    vocab_path = args.vocab if args.vocab != '' else None
    label_path = args.labels if args.labels != '' else None
    gold_props_path = args.gold if args.gold != '' else None
    
    print ('Task: {}'.format(args.task))
    
    data = TaggerData(config,
                        *reader.get_srl_data(config, args.train, args.dev, vocab_path, label_path))
    
    batched_dev_data = data.get_development_data(batch_size=config.dev_batch_size)
    print ('Dev data has {} batches.'.format(len(batched_dev_data)))

Task: srl
(['In', 'recent', 'years', ',', 'advanced', 'education', 'for', 'professionals', 'has', 'become', 'a', 'hot', 'topic', 'in', 'the', 'business', 'community', '.'], 5, ['O', 'O', 'O', 'O', 'B-ARGM-ADJ', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
(['President', 'Chen', 'Travels', 'Abroad'], 2, ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARGM-ADV'])


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting features
Extraced 43852 words and 129 tags
Max training sentence length: 210
Max development sentence length: 275
Dev data has 71 batches.
Data loading duration was 0:00:21.


In [15]:
data.train_sents[0][0]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 14, 15, 16, 17]

In [16]:
" ".join([data.word_dict.idx2str[i] for i in data.train_sents[0][0]])

'in recent years , advanced education for professionals has become a hot topic in the business community .'

In [17]:
data.train_sents[0][1]

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
data.train_sents[0][2]

[0, 0, 0, 0, 1, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [19]:
" ".join([data.label_dict.idx2str[i] for i in data.train_sents[0][2]])

'O O O O B-ARGM-ADJ B-V B-ARG1 I-ARG1 O O O O O O O O O O'

In [20]:
data.label_dict.idx2str[:3]

['O', 'B-ARGM-ADJ', 'B-V']

In [21]:
with Timer('Preparation'):
    if not os.path.isdir(args.model):
      print ('Directory {} does not exist. Creating new.'.format(args.model))
      os.makedirs(args.model)
    else:
      if len(os.listdir(args.model)) > 0:
        print ('[WARNING] Log directory {} is not empty, previous checkpoints might be overwritten'
             .format(args.model))
    shutil.copyfile(args.config, os.path.join(args.model, 'config'))
    # Save word and label dict to model directory.
    data.word_dict.save(os.path.join(args.model, 'word_dict'))
    data.label_dict.save(os.path.join(args.model, 'label_dict'))
    writer = open(os.path.join(args.model, 'checkpoints.tsv'), 'w')
    writer.write('step\tdatetime\tdev_loss\tdev_accuracy\tbest_dev_accuracy\n')

[WARNING] Log directory conll2012_small_model is not empty, previous checkpoints might be overwritten
Preparation duration was 0:00:00.


In [22]:
args.model

'conll2012_small_model'

In [23]:
train_data = data.get_training_data(include_last_batch=True)

Extracted 252315 samples and 3154 batches.


In [24]:
len(train_data[0][0])

80

In [25]:
with Timer('Building model'):
#     model = BiLSTMTaggerModel(data, config=config)  
#     for param in model.params:
#       print(param, param.name, param.shape.eval())
#     loss_function = model.get_loss_function()
#     eval_function = model.get_eval_function()
    print("cc")

cc
Building model duration was 0:00:00.


In [36]:
torch.tensor(1)

tensor(1)

In [27]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti'